In [ ]:
# 5 mM P. simiae Data

# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt

# Load tab-separated growth data file
df = pd.read_csv('growth1107.tsv', sep='\t')

# Filter columns that start with 's' (assumed to be timepoints)
growth_raw = df.filter(regex="^s")

# Extract numeric timepoint values from column names (e.g., s10 → 10)
timepoints_extracted = growth_raw.columns.str.extract(r's(\d+)')[0]

# Keep only columns with valid timepoint names
valid_col_names = growth_raw.columns[timepoints_extracted.notna()]

# Convert extracted timepoint strings to integers
timepoints = timepoints_extracted.dropna().astype(int)

# Select valid timepoint columns from original DataFrame
valid_growth = growth_raw[valid_col_names]

# Rename columns with integer timepoint values
valid_growth.columns = timepoints.values

# Transpose the DataFrame to have timepoints as rows, wells as columns
growth = valid_growth.T
growth.columns = df['Well']

# Create subplots: 8 rows × 12 columns (fits 96-well plate)
fig, axes = plt.subplots(8, 12, sharex=True, sharey=True, figsize=(40, 30))

# Adjust spacing between subplots and margins
fig.subplots_adjust(left=0.08, bottom=0.08, right=0.98, top=0.95, wspace=0.6, hspace=0.8)

# Add global x-axis label
fig.text(0.5, 0.04, 'Time (s)', ha='center', fontsize=18)

# Add global y-axis label
fig.text(0.04, 0.5, 'OD600', va='center', rotation='vertical', fontsize=18)

# Loop over each subplot and plot the corresponding growth curve
for i, ax in enumerate(fig.axes):
    if i >= len(df):
        break  # avoid indexing errors if there are fewer wells than subplots

    # Extract well ID and condition metadata
    well = df.at[i, 'Well']
    condition = str(df.at[i, 'Condition_1'])
    concentration = str(df.at[i, 'Concentration_1'])
    units = str(df.at[i, 'Units_1'])

    # Format label with concentration, units, and condition
    label = f"{concentration} {units} {condition}" if condition != 'None' else 'None'

    # Plot the growth curve for the current well
    ax.plot(growth[well])

    # Annotate the plot with well ID and condition label
    ax.annotate(f"{well}\n{label}", xy=(0.05, 0.8), xycoords='axes fraction', fontsize=9)

    # Set y-axis limit for consistency
    ax.set_ylim(0, 0.5)

# Save plot to PNG and PDF with high resolution and tight layout
plt.savefig("growth_curves_full.png", dpi=300, bbox_inches='tight') 
plt.savefig("growth_curves_full.pdf", bbox_inches='tight')          

# Display the full figure
plt.show()
